In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix, r2_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from typing import List
from itertools import combinations
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [35]:
import pandas as pd

df = pd.read_csv("lungcancer.csv", sep=",")
df


,GENDER;AGE;SMOKING;YELLOW_FINGERS;ANXIETY;PEER_PRESSURE;CHRONIC DISEASE;FATIGUE ;ALLERGY ;WHEEZING;ALCOHOL CONSUMING;COUGHING;SHORTNESS OF BREATH;SWALLOWING DIFFICULTY;CHEST PAIN;LUNG_CANCER
0,M;69;1;2;2;1;1;2;1;2;2;2;2;2;2;YES
1,M;74;2;1;1;1;2;2;2;1;1;1;2;2;2;YES
2,F;59;1;1;1;2;1;2;1;2;1;2;2;1;2;NO
3,M;63;2;2;2;1;1;1;1;1;2;1;1;2;2;NO
4,F;63;1;2;1;1;1;1;1;2;1;2;2;1;1;NO
...,...
304,F;56;1;1;1;2;2;2;1;1;2;2;2;2;1;YES
305,M;70;2;1;1;1;1;2;2;2;2;2;2;1;2;YES
306,M;58;2;1;1;1;1;1;2;2;2;2;1;1;2;YES
307,M;67;2;1;2;1;1;2;2;1;2;2;2;1;2;YES


In [36]:
import pandas as pd
import re

df = pd.read_csv("lungcancer.csv", sep=";")

with open("age.txt", "r") as file:
    age_intervals = re.findall(r'\[(\d+),(\d+)\)', file.read())

age_intervals = [(int(start), int(end)) for start, end in age_intervals]

def generalize_age(age):
    for interval in age_intervals:
        if age in range(interval[0], interval[1] + 1):
            return f"{interval[0]}-{interval[1]}"
    return "Unknown"

df['AGE_GROUP'] = df['AGE'].apply(generalize_age)

df.head(20)


,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER,AGE_GROUP
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES,50-75
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES,50-75
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO,50-75
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO,50-75
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO,50-75
5,F,75,1,2,1,1,2,2,2,2,1,2,2,1,1,YES,50-75
6,M,52,2,1,1,1,1,2,1,2,2,2,2,1,2,YES,50-75
7,F,51,2,2,2,2,1,2,2,1,1,1,2,2,1,YES,50-75
8,F,68,2,1,2,1,1,2,1,1,1,1,1,1,1,NO,50-75
9,M,53,2,2,2,2,2,1,2,1,2,1,1,2,2,YES,50-75


In [1]:
import pandas as pd
import re

df = pd.read_csv("lungcancer.csv", sep=";")

with open("age.txt", "r") as file:
    age_intervals = re.findall(r'\[(\d+),(\d+)\)', file.read())

age_intervals = [(int(start), int(end)) for start, end in age_intervals]

def generalize_age(age):
    for interval in age_intervals:
        if age >= interval[0] and age <= interval[1]:
            return f"{interval[0]}-{interval[1]}"
    return "Unknown"

df['AGE'] = df['AGE'].apply(generalize_age)

df.to_csv("lungcancer_age_generalized.csv", sep=";", index=False)

df.head(20)


,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,50-75,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,50-75,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,50-75,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,50-75,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,50-75,1,2,1,1,1,1,1,2,1,2,2,1,1,NO
5,F,50-75,1,2,1,1,2,2,2,2,1,2,2,1,1,YES
6,M,50-75,2,1,1,1,1,2,1,2,2,2,2,1,2,YES
7,F,50-75,2,2,2,2,1,2,2,1,1,1,2,2,1,YES
8,F,50-75,2,1,2,1,1,2,1,1,1,1,1,1,1,NO
9,M,50-75,2,2,2,2,2,1,2,1,2,1,1,2,2,YES
